encoder transformer层的linear1层（etl1）替换为近似矩阵乘法

In [28]:
import numpy as np
import os
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [29]:
# root_path = '/data/hdr/csi_transformer/pic/'
# print(root_path)
# root_path2 = '/'.join(root_path.split("/")[:-2])
# # sys.path.append(root_path)
# print(root_path2)

In [30]:
# method = METHOD_MITHRAL
method = METHOD_PQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE

In [31]:
linear_name = 'etl1'
feedback_bits = 256
ncodebooks = 32 # max:64
ncentroids = 256
if method == METHOD_MITHRAL:
    ncentroids = 16
train_sam_num = 3000 # 训练集样本数
test_sam_num = 1000 # 测试集样本数
batch_size = 32

In [32]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= ''
    featurepath_train= ''
    data_to_fcpath_test = ''
    featurepath_test = ''
elif host_name == 'jm-System-Product-Name':
    dir_train = os.path.join('/data/hdr/transformer_data/joined', 'train', 'f'+str(feedback_bits))
    dir_test = os.path.join('/data/hdr/transformer_data/joined', 'test', 'f'+str(feedback_bits))
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'ex_linear1in_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'ex_linear1_y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'ex_linear1out_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'ex_linear1in_test.npy'
    featurepath_test = 'ex_linear1out_test.npy'
    # y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dir_train")


weightpath = 'ex_linear1_w_f256.npy'
biaspath = 'ex_linear1_b_f256.npy'
dir_result = os.path.join(dir_result, method, "f%i" % feedback_bits, linear_name)
try:
    os.mkdir(dir_result)
except FileExistsError:
    pass 


In [33]:
est3 = mm.estFactory(X_path=data_to_fcpath_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method])


running method:  PQ
running kmeans in subspace 1/32... mse / {var(X_subs), var(X)}: 0.0104, 0.00393
running kmeans in subspace 2/32... mse / {var(X_subs), var(X)}: 0.00949, 0.00333
running kmeans in subspace 3/32... mse / {var(X_subs), var(X)}: 0.0154, 0.00563
running kmeans in subspace 4/32... mse / {var(X_subs), var(X)}: 0.0229, 0.0168
running kmeans in subspace 5/32... mse / {var(X_subs), var(X)}: 0.0137, 0.0236
running kmeans in subspace 6/32... mse / {var(X_subs), var(X)}: 0.0168, 0.0365
running kmeans in subspace 7/32... mse / {var(X_subs), var(X)}: 0.00916, 0.0219
running kmeans in subspace 8/32... mse / {var(X_subs), var(X)}: 0.0107, 0.0214
running kmeans in subspace 9/32... mse / {var(X_subs), var(X)}: 0.0131, 0.0108
running kmeans in subspace 10/32... mse / {var(X_subs), var(X)}: 0.0145, 0.00729
running kmeans in subspace 11/32... mse / {var(X_subs), var(X)}: 0.0128, 0.00362
running kmeans in subspace 12/32... mse / {var(X_subs), var(X)}: 0.0119, 0.00308
running kmeans in sub

In [34]:
# type(est3)

In [35]:
x_test = np.load(dir_test+'/'+data_to_fcpath_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [36]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, batch_size, -1, y_out_last.shape[-1])
print("y_out_last_re.shape: ", y_out_last_re.shape)
# np.save("LDPC_decoder_NET_testdata/" + snr + "nomul_matmul_yout_matmul", y_out_matmul)
# np.save(dir_result+'/'+method+'fc1_fb256_cb%i_ct%i.npy' % (ncodebooks, ncentroids), y_out_matmul)
np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, ncodebooks, ncentroids)), y_out_last_re)
# io.savemat(dir_result+'\\fc1_256.mat', {"NN_output_buffer": y_out_last})

[[-0.4879309  -0.45174497 -0.5377412  ... -0.35225588  0.9048168
  -0.4253752 ]
 [-0.5087167  -0.46837363 -0.46291226 ... -0.29821277  0.8881882
  -0.39627504]
 [-0.43388778 -0.43095917 -0.5127982  ... -0.33562723  0.9339169
  -0.4253752 ]
 ...
 [-0.31333008 -0.30208716 -0.4254978  ... -0.30236995  0.72190166
  -0.33391765]
 [-0.38400185 -0.37691608 -0.42134064 ... -0.13608345  0.76763046
  -0.37133208]
 [-0.25097266 -0.31040147 -0.18853958 ... -0.12361197  0.5597724
  -0.18841696]]
y_out_last.shape:  (1024000, 512)
y_out_last_re.shape:  (1000, 32, 32, 512)
